<a href="https://colab.research.google.com/github/hrishi483/ASR-using-LLMs-for-USR/blob/main/Fine_Tuning_using_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 10.8 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-jmzfkl58
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-jmzfkl58
  Resolved https://github.com/huggingface/transformers to commit a09130feee8138483e1ce5fb7997e20c7598a12b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 M

In [2]:
!sudo apt-get install unzip


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


Zip your custom dataset an example of zip file format is shown on github with the name Hindi_Commonvoice_data-20230816T105523Z-001.zip

In [4]:
!unzip /content/Hindi_Commonvoice_data-20230816T105523Z-001.zip

Archive:  /content/Hindi_Commonvoice_data-20230816T105523Z-001.zip
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37873511.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37754363.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37754360.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37895003.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37890679.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37873512.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37817520.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37817524.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37890680.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37890682.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37803369.txt  
  inflating: Hindi_Commonvoice_

**Load the train data**

In [14]:
import glob
import re
path = glob.glob('/content/Hindi_Commonvoice_data/data/train/texts/*.txt')
audio_path = '/content/Hindi_Commonvoice_data/data/train/audios'
data=[]
with open('train.csv', 'w') as g:
  g.write(f"text,path,audio\n")
  for paths in path:
    name = paths.split('/')[-1]
    name_audio = name.split('.')[0] + '.mp3'
    with open(paths, 'r') as f:
      for file in f:
        text = file.strip().lower()
        res = re.sub(r'[^\w\s]', '', text)
        g.write(f"{res},{audio_path},{audio_path}/{str(name_audio)}\n")

**Load the val data**

In [11]:
import glob
import re
path = glob.glob('/content/Hindi_Commonvoice_data/data/val/texts/*.txt')
audio_path = '/content/Hindi_Commonvoice_data/data/val/audios'
data=[]
with open('val.csv', 'w') as g:
  g.write(f"text,path,audio\n")
  for paths in path:
    name = paths.split('/')[-1]
    name_audio = name.split('.')[0] + '.mp3'
    with open(paths, 'r') as f:
      for file in f:
        text = file.strip().lower()
        res = re.sub(r'[^\w\s]', '', text)
        g.write(f"{res},{audio_path},{audio_path}/{str(name_audio)}\n")

In [12]:
path

['/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37803368.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37890683.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37890682.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37754363.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37873511.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37803371.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37754361.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37868076.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37895003.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37695873.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37695872.txt',
 '/content/Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37868079.txt',
 '/content/Hindi_Commonvoice_data/data/v

In [13]:
!cat train.csv

cat: train.csv: No such file or directory


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
from datasets import Dataset, Audio, Value, Features
from datasets import load_dataset

features = Features(
    {
        "text": Value("string"),
        'path': Value('string'),
        "audio": Audio(sampling_rate=16000)
    }
)
sample_data = load_dataset(
    'csv', data_files={
        'train': '/content/train.csv',
        'valid': '/content/val.csv'
    }
)


sample_data = sample_data.cast(features)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Casting the dataset:   0%|          | 0/149 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/33 [00:00<?, ? examples/s]

# Load the tokenizers from whispher for whichever language you want to fine tune on.
You can also load your own tokenizers

In [16]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="Assamese", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="Assamese", task="transcribe")


Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
print(sample_data["train"][0])

{'text': 'अत पर पहच जनत क आकरश', 'path': '/content/Hindi_Commonvoice_data/data/train/audios', 'audio': {'path': '/content/Hindi_Commonvoice_data/data/train/audios/common_voice_hi_37630567.mp3', 'array': array([-5.58793545e-09, -1.49011612e-08, -3.72529030e-09, ...,
        1.64654921e-08,  1.17188392e-08,  3.61137609e-08]), 'sampling_rate': 16000}}


# Audio Preprocessing
Before applying the model on the data
- compute log-Mel for input audio
- encode text to labels

In [20]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [19]:
custom_data = sample_data.map(prepare_dataset, remove_columns=sample_data.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/149 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/33 [00:00<?, ? examples/s]

In [21]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [22]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [23]:
import evaluate

metric = evaluate.load("wer")

In [24]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [25]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

In [26]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [35]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [36]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=custom_data["train"],
    eval_dataset=custom_data["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [37]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
100,0.559100,0.771864,80.379747
200,0.033100,0.946820,66.772152
300,0.004800,1.105487,67.721519
400,0.000900,1.172862,64.240506


Step,Training Loss,Validation Loss,Wer
100,0.559100,0.771864,80.379747
200,0.033100,0.946820,66.772152
300,0.004800,1.105487,67.721519
400,0.000900,1.172862,64.240506
500,0.000600,1.219150,67.088608
600,0.000400,1.252677,83.227848
700,0.000300,1.276438,69.936709
